In [1]:
!git clone https://github.com/hiteshmishra708/python-chatbot.git

Cloning into 'python-chatbot'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 20 (delta 3), reused 17 (delta 3), pack-reused 0
Receiving objects: 100% (20/20), 478.91 KiB | 2.46 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
%cd python-chatbot

/content/python-chatbot


In [3]:
!ls

chatbot_model.h5  classes.pkl	README.md	  train_bot.py
chatgui.py	  intents.json	requirements.txt  words.pkl


In [4]:
!cat intents.json

{
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hi there",
                "How are you",
                "Is anyone there?",
                "Hey",
                "Hola",
                "Hello",
                "Good day"
            ],
            "responses": [
                "Hi there, how can I help?"
            ],
            "context": [
                ""
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "Bye",
                "See you later",
                "Goodbye",
                "Nice chatting to you, bye",
                "Till next time"
            ],
            "responses": [
                "Bye! Have a nice day"
            ],
            "context": [
                ""
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thanks",
                "Thank you",
                "That's helpful",
    

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import json
import pickle

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
print(intents)

# intents: opciones de conversación (lo que el chatbot contestará al usuario)
# patterns: interacciones del usuario (lo que pensamos que puede contestar)
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenizamos
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # añadimos las palabras
        documents.append((w, intent['tag']))

        # añadimos las clases a nuestra lista
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

# preparamos los datos para entrenar la red
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # bag of words
    bag = []
    # lista de tokens
    pattern_words = doc[0]
    # lemmatizacion de los tokens
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])


training = np.array(training, dtype="object")
# creamos los conjuntos de entrenamiento y test: X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

# creación del modelo
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# entrenamos y guardamos el modelo
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['Bye! Have a nice day'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you"], 'context': ['']}]}


Epoch 1/300
4/4 [==============================] - 1s 6ms/step - loss: 1.1352 - accuracy: 0.2941
Epoch 2/300
4/4 [==============================] - 0s 7ms/step - loss: 1.1230 - accuracy: 0.3529
Epoch 3/300
4/4 [==============================] - 0s 5ms/step - loss: 1.1322 - accuracy: 0.2941
Epoch 4/300
4/4 [==============================] - 0s 4ms/step - loss: 1.1544 - accuracy: 0.2941
Epoch 5/300
4/4 [==============================] - 0s 4ms/step - loss: 1.0494 - accuracy: 0.5294
Epoch 6/300
4/4 [==============================] - 0s 4ms/step - loss: 0.9530 - accuracy: 0.5294
Epoch 7/300
4/4 [==============================] - 0s 4ms/step - loss: 0.9667 - accuracy: 0.4706
Epoch 8/300
4/4 [==============================] - 0s 4ms/step - loss: 0.9567 - accuracy: 0.5882
Epoch 9/300
4/4 [==============================] - 0s 4ms/step - loss: 0.8026 - accuracy: 0.7647
Epoch 10/300
4/4 [==============================] - 0s 4ms/step - loss: 0.8306 - accuracy: 0.7059
Epoch 11/300
4/4 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import nltk, json, random, pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# preprocesamos la entrada
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# creación del BoW
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def calcula_pred(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def respuesta(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def responde(msg):
    ints = calcula_pred(msg, model)
    res = respuesta(ints, intents)
    return res

entrada = ''
print('Welcome! Type "exit" to quit.')

while entrada != 'exit':
    entrada = str(input(""))
    res = responde(entrada)
    print('AI:' + res)

Welcome! Type "exit" to quit.
Hola
1/1 [==============================] - 0s 89ms/step
AI:Hi there, how can I help?
Que tal?
1/1 [==============================] - 0s 31ms/step
AI:Hi there, how can I help?


KeyboardInterrupt: Interrupted by user

### Recursos:
- https://planetachatbot.com/tutorial-chatbot-usando-nltk-keras/
- https://github.com/hiteshmishra708/python-chatbot/
- https://www.pycodemates.com/2021/11/build-a-AI-chatbot-using-python-and-deep-learning.html